# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 24 2022 10:49AM,237961,16,EMERSA-8056939,"Emersa Waterbox, LLC",Released
1,May 24 2022 10:20AM,237958,10,LOA157337,Beach Products Inc,Executing
2,May 24 2022 10:13AM,237957,16,SHL-8056935,"SHL Pharma, LLC",Released
3,May 24 2022 10:10AM,237955,10,0085701275,ISDIN Corporation,Executing
4,May 24 2022 10:10AM,237955,10,0085701284,ISDIN Corporation,Executing
5,May 24 2022 10:10AM,237955,10,0085701285,ISDIN Corporation,Executing
6,May 24 2022 10:10AM,237955,10,0085701283,ISDIN Corporation,Executing
7,May 24 2022 10:10AM,237955,10,0085701286,ISDIN Corporation,Executing
8,May 24 2022 10:10AM,237955,10,0085701290,ISDIN Corporation,Executing
9,May 24 2022 10:10AM,237955,10,0085701289,ISDIN Corporation,Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,237955,Executing,45
39,237955,Released,4
40,237957,Released,1
41,237958,Executing,1
42,237961,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
237954,NaN,4.0
237955,45.0,4.0
237957,NaN,1.0
237958,1.0,NaN
237961,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
237673,0.0,1.0
237701,0.0,1.0
237861,6.0,17.0
237864,0.0,11.0
237866,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
237673,0,1
237701,0,1
237861,6,17
237864,0,11
237866,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,237673,0,1
1,237701,0,1
2,237861,6,17
3,237864,0,11
4,237866,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,237673,,1
1,237701,,1
2,237861,6,17
3,237864,,11
4,237866,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 24 2022 10:49AM,237961,16,"Emersa Waterbox, LLC"
1,May 24 2022 10:20AM,237958,10,Beach Products Inc
2,May 24 2022 10:13AM,237957,16,"SHL Pharma, LLC"
3,May 24 2022 10:10AM,237955,10,ISDIN Corporation
52,May 24 2022 10:07AM,237952,10,ISDIN Corporation
87,May 24 2022 10:07AM,237951,10,ISDIN Corporation
122,May 24 2022 10:05AM,237954,10,ISDIN Corporation
126,May 24 2022 10:02AM,237864,20,"ACI Healthcare USA, Inc."
137,May 24 2022 9:48AM,237950,10,ISDIN Corporation
139,May 24 2022 9:47AM,237949,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,May 24 2022 10:49AM,237961,16,"Emersa Waterbox, LLC",,1
1,May 24 2022 10:20AM,237958,10,Beach Products Inc,1,
2,May 24 2022 10:13AM,237957,16,"SHL Pharma, LLC",,1
3,May 24 2022 10:10AM,237955,10,ISDIN Corporation,45,4
4,May 24 2022 10:07AM,237952,10,ISDIN Corporation,,35
5,May 24 2022 10:07AM,237951,10,ISDIN Corporation,,35
6,May 24 2022 10:05AM,237954,10,ISDIN Corporation,,4
7,May 24 2022 10:02AM,237864,20,"ACI Healthcare USA, Inc.",,11
8,May 24 2022 9:48AM,237950,10,ISDIN Corporation,,2
9,May 24 2022 9:47AM,237949,10,ISDIN Corporation,12,22


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,May 24 2022 10:49AM,237961,16,"Emersa Waterbox, LLC",1,
1,May 24 2022 10:20AM,237958,10,Beach Products Inc,,1
2,May 24 2022 10:13AM,237957,16,"SHL Pharma, LLC",1,
3,May 24 2022 10:10AM,237955,10,ISDIN Corporation,4,45
4,May 24 2022 10:07AM,237952,10,ISDIN Corporation,35,
5,May 24 2022 10:07AM,237951,10,ISDIN Corporation,35,
6,May 24 2022 10:05AM,237954,10,ISDIN Corporation,4,
7,May 24 2022 10:02AM,237864,20,"ACI Healthcare USA, Inc.",11,
8,May 24 2022 9:48AM,237950,10,ISDIN Corporation,2,
9,May 24 2022 9:47AM,237949,10,ISDIN Corporation,22,12


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,May 24 2022 10:49AM,237961,16,"Emersa Waterbox, LLC",1,
1,May 24 2022 10:20AM,237958,10,Beach Products Inc,,1
2,May 24 2022 10:13AM,237957,16,"SHL Pharma, LLC",1,
3,May 24 2022 10:10AM,237955,10,ISDIN Corporation,4,45
4,May 24 2022 10:07AM,237952,10,ISDIN Corporation,35,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,May 24 2022 10:49AM,237961,16,"Emersa Waterbox, LLC",1.0,NaN
1,May 24 2022 10:20AM,237958,10,Beach Products Inc,NaN,1.0
2,May 24 2022 10:13AM,237957,16,"SHL Pharma, LLC",1.0,NaN
3,May 24 2022 10:10AM,237955,10,ISDIN Corporation,4.0,45.0
4,May 24 2022 10:07AM,237952,10,ISDIN Corporation,35.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,May 24 2022 10:49AM,237961,16,"Emersa Waterbox, LLC",1.0,0.0
1,May 24 2022 10:20AM,237958,10,Beach Products Inc,0.0,1.0
2,May 24 2022 10:13AM,237957,16,"SHL Pharma, LLC",1.0,0.0
3,May 24 2022 10:10AM,237955,10,ISDIN Corporation,4.0,45.0
4,May 24 2022 10:07AM,237952,10,ISDIN Corporation,35.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1665669,102.0,58.0
12,237883,0.0,1.0
15,1189664,49.0,44.0
16,951698,4.0,9.0
17,237924,1.0,0.0
18,951237,17.0,6.0
19,951739,4.0,0.0
20,1427361,62.0,15.0
22,475773,0.0,2.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1665669,102.0,58.0
1,12,237883,0.0,1.0
2,15,1189664,49.0,44.0
3,16,951698,4.0,9.0
4,17,237924,1.0,0.0
5,18,951237,17.0,6.0
6,19,951739,4.0,0.0
7,20,1427361,62.0,15.0
8,22,475773,0.0,2.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,102.0,58.0
1,12,0.0,1.0
2,15,49.0,44.0
3,16,4.0,9.0
4,17,1.0,0.0
5,18,17.0,6.0
6,19,4.0,0.0
7,20,62.0,15.0
8,22,0.0,2.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,102.0
1,12,Released,0.0
2,15,Released,49.0
3,16,Released,4.0
4,17,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,17,18,19,20,22
Status,,,,,,,,,
Executing,58.0,1.0,44.0,9.0,0.0,6.0,0.0,15.0,2.0
Released,102.0,0.0,49.0,4.0,1.0,17.0,4.0,62.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,17,18,19,20,22
0,Executing,58.0,1.0,44.0,9.0,0.0,6.0,0.0,15.0,2.0
1,Released,102.0,0.0,49.0,4.0,1.0,17.0,4.0,62.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,17,18,19,20,22
0,Executing,58.0,1.0,44.0,9.0,0.0,6.0,0.0,15.0,2.0
1,Released,102.0,0.0,49.0,4.0,1.0,17.0,4.0,62.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()